In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "tesi2"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/tesi2

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
!pip install captum

     |████████████████████████████████| 4.4MB 9.4MB/s 


In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torchvision import *

from captum.attr import IntegratedGradients
modelName = 'denseNet121'
use_cuda = True

models = {
    'AlexNet': models.alexnet,
    'ResNet18': models.resnet18,
    'denseNet121': models.densenet121,
    'vgg16':models.vgg16,
    'googlenet':models.googlenet,
    'inception':models.inception_v3,
    'shufflenet': models.shufflenet_v2_x1_0
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = models[modelName]()

if modelName == 'ResNet18':
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    if use_cuda: model.fc = model.fc.cuda()

elif modelName == 'AlexNet':
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 2)
    if use_cuda: model.classifier[6] = model.classifier[6].cuda()

elif modelName == 'denseNet121':
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 3)
    if use_cuda: model.classifier = model.classifier.cuda()

model.load_state_dict(torch.load("models/denseNet121Base.pt"))
model = model.cuda() if device else model
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
import matplotlib.pyplot as plt
path = 'roi_train/benign/'


ig = IntegratedGradients(model)
baseline = torch.zeros(1,3,400,400).cuda()
print(baseline.shape)

input = plt.imread(path + '00083_RIGHT_CC.png')[:,:,:3]
input = torch.from_numpy(input).permute(2,0,1).unsqueeze(0).cuda()
print(input.shape)

attributions, delta = ig.attribute(input, baseline, target=1, return_convergence_delta=True)
# print('IG Attributions:', attributions)
# print('Convergence Delta:', delta)

torch.Size([1, 3, 400, 400])
torch.Size([1, 3, 400, 400])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: ignored

In [ ]:
scores = attributions.squeeze(0)

scores = torch.sum(torch.abs(scores), axis=0).cpu()
scores /= torch.max(scores)
print(torch.max(scores))
input = input.cpu().squeeze(0).permute(1,2,0)
plt.imshow(input)
plt.show()
plt.imshow(scores)
plt.show()